In [42]:
# Imports
import datetime, json, pytz
import os
import pandas as pd
from copy import copy
from django.apps import apps as django_apps
from django.db.models.signals import post_save
from django.core.exceptions import ValidationError
from flourish_caregiver.helper_classes import SequentialCohortEnrollment
from flourish_caregiver.helper_classes.schedule_dict import child_schedule_dict, caregiver_schedule_dict
from flourish_child.models import Appointment as ChildAppointment
from edc_appointment.models import Appointment as CaregiverAppointment
from edc_appointment.constants import NEW_APPT
from edc_base.utils import age, get_utcnow
from flourish_caregiver.helper_classes.onschedule_helper import OnScheduleHelper
from edc_visit_schedule.site_visit_schedules import site_visit_schedules
from edc_reference.site import SiteReferenceConfigError

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [43]:
def update_forms_visit(forms, visit=None, new_visit=None, form_type='crf'):
    visit_attr = 'child_visit' if isinstance(visit, ChildVisit) else 'maternal_visit'
    for form in forms:
        kwargs = {f'{visit_attr}': visit,}
        model_cls = django_apps.get_model(form.model)
        if form_type == 'requisition':
            kwargs.update({'panel__name': form.panel_name})
            post_save.disconnect(dispatch_uid='senaite_sample_create_on_post_save')
        try:
            model_obj = model_cls.objects.get(**kwargs)
        except model_cls.DoesNotExist:
            continue
        else:
            print(model_obj)
            if visit_attr == 'child_visit':
                model_obj.child_visit = new_visit
            else:
                model_obj.maternal_visit = new_visit
            try:
                model_obj.save()
            except SiteReferenceConfigError:
                continue

In [44]:
def delete_old_metadata(visit):
    reqs = RequisitionMetadata.objects.filter(
        subject_identifier=visit.subject_identifier,
        visit_code=visit.visit_code,
        schedule_name=visit.schedule_name,)
    reqs.filter(entry_status='keyed').update(entry_status='required')
    reqs.delete()
    
    crfs = CrfMetadata.objects.filter(
        subject_identifier=visit.subject_identifier,
        visit_code=visit.visit_code,
        schedule_name=visit.schedule_name,)
    crfs.filter(entry_status='keyed').update(entry_status='required')
    crfs.delete()

In [45]:
child_idx = 'B142-040990972-7-10'
idx = 'B142-040990972-7'
cohort = Cohort.objects.filter(subject_identifier=child_idx).latest('assign_datetime')


cohort_schedules = CohortSchedules.objects.filter(
    cohort_name=cohort.name, schedule_type='quarterly')
new_schedules = CohortSchedules.objects.filter(cohort_name=cohort.name,
                                               schedule_type='followup_quarterly')

for schedule in cohort_schedules:
    q_object = Q()
    subject_identifier = None
    appt_model_cls = None
    is_caregiver = False
    if schedule.onschedule_model.startswith('flourish_child'):
        subject_identifier = child_idx
        appt_model_cls = ChildAppointment
        visit_model_cls = ChildVisit
    else:
        is_caregiver = True
        subject_identifier = idx
        appt_model_cls = CaregiverAppointment
        visit_model_cls = MaternalVisit
        q_object = Q(child_subject_identifier=child_idx)

    model_cls = django_apps.get_model(schedule.onschedule_model)
    try:
        onschedule_obj = model_cls.objects.get(
            q_object,
            subject_identifier=subject_identifier,
            schedule_name=schedule.schedule_name)
    except model_cls.DoesNotExist:
        print(f'missing onschedule obj, {schedule.schedule_name}, for {subject_identifier}')
    else:
        q_schedule = None
        if is_caregiver:
            q_schedule = Q(
                onschedule_model__startswith='flourish_caregiver', child_count=schedule.child_count)
        else:
            q_schedule = Q(onschedule_model__startswith='flourish_child')
        try:
            new_schedule = new_schedules.get(q_schedule)
        except CohortSchedules.DoesNotExist:
            print(f'missing new onschedule obj, {schedule} {subject_identifier}')
        else:
            appts = appt_model_cls.objects.filter(
                subject_identifier=subject_identifier,
                schedule_name=onschedule_obj.schedule_name).exclude(appt_status=NEW_APPT)

            for appt in appts:
                try:
                    visit = visit_model_cls.objects.get(appointment=appt)
                except visit_model_cls.DoesNotExist:
                    print(f'visit does not exist {appt}')
                else:
                    new_code = f'3{appt.visit_code[1:]}'
                    if new_code.endswith('M'):
                        new_code = new_code.replace('M', '')
                    new_code += 'S'
 
                    try:
                        new_appt = appt_model_cls.objects.get(
                            subject_identifier=subject_identifier,
                            schedule_name=new_schedule.schedule_name,
                            visit_code=new_code,
                            visit_code_sequence=appt.visit_code_sequence)
                    except appt_model_cls.DoesNotExist:
                        print(f'no appt for visit code {new_code}')
                    else:
                        new_visit = visit_model_cls.objects.filter(appointment=new_appt)
                        if not new_visit.exists():
                            new_visit = copy(visit)
                            new_visit.pk = None
                            new_visit.visit_code = new_appt.visit_code
                            new_visit.appointment = new_appt
                            new_visit.schedule_name = new_appt.schedule_name
                            new_visit.visit_schedule_name = new_appt.visit_schedule_name
                            new_visit.save()
                        else:
                            new_visit = new_visit[0]

                        crfs = CrfMetadata.objects.filter(
                            subject_identifier=subject_identifier,
                            visit_code=visit.visit_code,
                            visit_code_sequence=visit.visit_code_sequence,
                            schedule_name=visit.schedule_name,
                            entry_status='KEYED')
                        print(visit, new_visit)
                        update_forms_visit(forms=crfs, visit=visit, new_visit=new_visit)

                        reqs = RequisitionMetadata.objects.filter(
                            subject_identifier=subject_identifier,
                            visit_code=visit.visit_code,
                            visit_code_sequence=visit.visit_code_sequence,
                            schedule_name=visit.schedule_name,
                            entry_status='KEYED')
                        update_forms_visit(forms=reqs, visit=visit, new_visit=new_visit, form_type='requisition')
                        delete_old_metadata(visit)

no appt for visit code 3002S
B142-040990972-7-10 2003.0 B142-040990972-7-10 3003S.0
missing onschedule obj, b_sec_quart3_schedule1, for B142-040990972-7
no appt for visit code 3002S
B142-040990972-7 2003M.0 B142-040990972-7 3003S.0
B142-040990972-7 2003M.0
missing onschedule obj, b_sec_quart2_schedule1, for B142-040990972-7


In [62]:
def remove_old_schedule_data(visit_cls, visit_attr, appt_cls, onsch):
    subject_identifier = onsch.subject_identifier
    appts = appt_cls.objects.filter(
        subject_identifier=subject_identifier, schedule_name=onsch.schedule_name)
    for appt in appts:
        try:
            visit = visit_cls.objects.get(appointment=appt)
        except visit_cls.DoesNotExist:
            continue
        else:
            crfs = CrfMetadata.objects.filter(
                subject_identifier=subject_identifier,
                visit_code=visit.visit_code,
                visit_code_sequence=visit.visit_code_sequence,
                schedule_name=visit.schedule_name,
                entry_status='KEYED')
            for crf in crfs:
                model_cls = django_apps.get_model(crf.model)
                try:
                    model_obj = model_cls.objects.get(**{f'{visit_attr}': visit})
                except model_cls.DoesNotExist:
                    continue
                else:
                    model_obj.delete()
            reqs = RequisitionMetadata.objects.filter(
                subject_identifier=subject_identifier,
                visit_code=visit.visit_code,
                visit_code_sequence=visit.visit_code_sequence,
                schedule_name=visit.schedule_name,
                entry_status='KEYED')
            for req in reqs:
                model_cls = django_apps.get_model(req.model)
                try:
                    model_obj = model_cls.objects.get(**{f'{visit_attr}': visit, 'panel__name': req.panel_name})
                except model_cls.DoesNotExist:
                    continue
                else:
                    model_obj.delete()
            delete_old_metadata(visit)
            visit.delete()
    appts.delete()

In [63]:
def remove_onsch_objs(onsch):
    ssh = SubjectScheduleHistory.objects.filter(
        subject_identifier=onsch.subject_identifier, schedule_name=onsch.schedule_name)
    if ssh.exists():
        ssh.delete()
    onsch.delete()

In [64]:
onsch = OnScheduleCohortBSecQuart.objects.get(subject_identifier=idx, child_subject_identifier=child_idx)
child_onsch = OnScheduleChildCohortBSecQuart.objects.get(subject_identifier=child_idx)

remove_old_schedule_data(MaternalVisit, 'maternal_visit', CaregiverAppointment, onsch)
remove_old_schedule_data(ChildVisit, 'child_visit', ChildAppointment, child_onsch)
remove_onsch_objs(onsch)
remove_onsch_objs(child_onsch)